<a href="https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/RuPERTa_base_finetuned_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to fine tune RuPERTa-base for POS downstream task

> Creator: [Manuel Romero](https://twitter.com/mrm8488)

In [1]:
!nvidia-smi

Thu May 14 21:15:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!git clone https://github.com/huggingface/transformers

In [0]:
!pip install -q ./transformers
!pip install -q tensorboardX
!pip install -q seqeval

In [0]:
!wget https://raw.githubusercontent.com/mrm8488/POS_es_processed/master/train.txt
!wget https://raw.githubusercontent.com/mrm8488/POS_es_processed/master/dev.txt
!wget https://raw.githubusercontent.com/mrm8488/POS_es_processed/master/labels.txt

In [0]:
!mkdir pos_dataset

In [0]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

In [32]:
!python3 preprocess.py train.txt "mrm8488/RuPERTa-base" 128 > /content/pos_dataset/train.txt
!python3 preprocess.py dev.txt "mrm8488/RuPERTa-base" 128 > /content/pos_dataset/dev.txt

2020-05-14 22:27:12.952453: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-14 22:27:31.869175: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1


In [0]:
!pip install -q wandb

In [0]:
!wandb login

In [0]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir /content/runs

In [0]:
%env WANDB_PROJECT="fine-tune-RuPERTa-spanish-POS"

In [0]:
!python /content/transformers/examples/token-classification/run_ner.py --data_dir /content/pos_dataset \
--model_type roberta \
--labels /content/labels.txt \
--model_name_or_path mrm8488/RuPERTa-base \
--output_dir model_output \
--max_seq_length 128 \
--num_train_epochs 10 \
--per_gpu_train_batch_size 64 \
--per_gpu_eval_batch_size 32 \
--overwrite_output_dir \
--save_steps 1000 \
--do_train \
--do_eval

In [0]:
!mkdir /content/RuPERTa-base-finetuned-pos

In [0]:
!mv /content/model_output/* /content/RuPERTa-base-finetuned-pos

In [0]:
!rm -rf /content/RuPERTa-base-finetuned-pos/checkpoint-*

In [0]:
!transformers-cli login

In [0]:
!transformers-cli upload ./RuPERTa-base-finetuned-pos

## Test the model

In [0]:
from transformers import pipeline

In [24]:
nlp_ner = pipeline(
    "ner",
    model="mrm8488/RuPERTa-base-finetuned-pos",
    tokenizer="mrm8488/RuPERTa-base-finetuned-pos")

text = 'Mis amigos están pensando viajar a Londres este verano.'

nlp_ner(text)[1:-1]

[{'entity': 'NUM', 'score': 0.9980951547622681, 'word': 'ĠMis'},
 {'entity': 'PRON', 'score': 0.5426339507102966, 'word': 'Ġamigos'},
 {'entity': 'AUX', 'score': 0.9093925952911377, 'word': 'ĠestÃ¡n'},
 {'entity': 'ADV', 'score': 0.6312834620475769, 'word': 'Ġpensando'},
 {'entity': 'VERB', 'score': 0.3273673355579376, 'word': 'Ġviajar'},
 {'entity': 'ADP', 'score': 0.9884780049324036, 'word': 'Ġa'},
 {'entity': 'NOUN', 'score': 0.4820754826068878, 'word': 'ĠLondres'},
 {'entity': 'DET', 'score': 0.820732593536377, 'word': 'Ġeste'},
 {'entity': 'NOUN', 'score': 0.8927345275878906, 'word': 'Ġverano'},
 {'entity': 'PUNCT', 'score': 0.999255359172821, 'word': '.'}]